In [16]:
from datasets import load_dataset

emotion_dataset = load_dataset("cardiffnlp/super_tweeteval", "tweet_emotion")
sentiment_dataset = load_dataset("cardiffnlp/super_tweeteval", "tweet_sentiment")

In [23]:
# EMOTION

label2emotion = {0 : "anger", 1 : "anticipation", 2 : "disgust", 3 : "fear", 
                 4 : "joy", 5 : "love", 6 : "optimism", 7 : "pessimism", 8 : "sadness", 9 : "surprise", 10 : "trust"}

def manipulate_emotion_labels(label):
    ris = []
    for i,e in enumerate(label):
        if e == 1:
            ris.append(label2emotion[i])
    return ris

emotion_train_dataset = emotion_dataset['train']
emotion_test_dataset = emotion_dataset['test']
emotion_validation_dataset = emotion_dataset['validation']

emotion_train_texts = emotion_train_dataset['text']
emotion_train_labels = emotion_train_dataset['gold_label_list']

for i in range(15,20):
    print(emotion_train_texts[i])
    print(manipulate_emotion_labels(emotion_train_labels[i]))


People you need to look up the definition of protest. What you are doing is not protesting is called vandalism. #angry #stop
['anger', 'disgust', 'sadness']
@user Look at those teef! #growl
['anger', 'disgust', 'fear']
Star trek online has a update to download oh fuming yay
['anger', 'disgust', 'joy', 'sadness']
The bitter the battle, the sweeter the victory...
['joy', 'optimism']
i cant stop. i finished - dejected. luckily no one is in the bathroom. so i go to a stall and wait until my pants are dry.
['anger', 'disgust', 'sadness']


In [27]:
print(sentiment_dataset)
sentiment_train_dataset = sentiment_dataset['train']
sentiment_test_dataset = sentiment_dataset['test']
sentiment_validation_dataset = sentiment_dataset['validation']

sentiment_train_texts = sentiment_train_dataset['text']
sentiment_train_labels = sentiment_train_dataset['gold_label']
#sentiment_train_targets = sentiment_train_dataset['target']

print(sentiment_train_texts[:5])
print(sentiment_train_labels[:5])
#print(sentiment_train_targets[:5])


DatasetDict({
    train: Dataset({
        features: ['gold_label', 'text', 'target'],
        num_rows: 26632
    })
    test: Dataset({
        features: ['gold_label', 'text', 'target'],
        num_rows: 12379
    })
    validation: Dataset({
        features: ['gold_label', 'text', 'target'],
        num_rows: 4000
    })
})
["dear @Microsoft the newOoffice for Mac is great and all, but no Lync update? C'mon.", "@Microsoft how about you make a system that doesn't eat my friggin discs. This is the 2nd time this has happened and I am so sick of it!", "I may be ignorant on this issue but... should we celebrate @user parental leave changes? Doesn't the gender divide suggest... (1/2)", 'Thanks to @user I just may be switching over to @user', 'If I make a game as a #windows10 Universal App. Will #xboxone owners be able to download and play it in November? @majornelson @Microsoft']
[1, 0, 1, 1, 2]
['@microsoft', '@microsoft', '@microsoft', '@microsoft', '@microsoft']


In [31]:
import json
from tqdm import tqdm

sentiment_prompt = """What is the sentiment of this text? \nText: {text} \nOptions: [ "strongly negative", "negative", "negative or neutral", "positive", "strongly positive"] \nAnswer: {answer}"""
emotion_prompt = """Which emotions from the options below are expressed in the following text? \nText: {text} \nOptions: [ "anger", "anticipation", "disgust", "fear", "joy", "love", "optimism", "pessimism", "sadness", "surprise", "trust" ] \nAnswer: {answer}"""

label2emotion = {0 : "anger", 1 : "anticipation", 2 : "disgust", 3 : "fear", 
                 4 : "joy", 5 : "love", 6 : "optimism", 7 : "pessimism", 8 : "sadness", 9 : "surprise", 10 : "trust"}
label2sentiment = {0 : "strongly negative", 1 : "negative", 2 : "negative or neutral", 3 : "positive", 4 : "strongly positive"}

def manipulate_emotion_labels(label):
    ris = []
    for i,e in enumerate(label):
        if e == 1:
            ris.append(label2emotion[i])
    return ", ".join(ris)


def generate_finetuning_dataset(dataset_type, texts, labels):

    json_data = []
    with open(f"training_{dataset_type}.json", "w") as fw_json:
        for instance_data, instance_gold in tqdm(zip(texts, labels), total=len(labels)):
            if dataset_type=="emotion":
                answer = manipulate_emotion_labels(instance_gold)
            else:
                answer = label2sentiment[instance_gold]
            
            prompt_template = emotion_prompt if dataset_type=="emotion" else sentiment_prompt
            prompt = prompt_template.format(
                    text=instance_data,
                    answer=answer)
            json_elem = {"prompt":prompt}
            json_data.append(json_elem)
        json.dump(json_data, fw_json, indent=4)
        
generate_finetuning_dataset("emotion", emotion_train_texts, emotion_train_labels)

100%|██████████| 6838/6838 [00:00<00:00, 517860.19it/s]
